## Set up

In [1]:
from google.cloud import bigquery

In [2]:
bq_client = bigquery.Client()
dataset_id = 'bqml-sandbox-396011.BQ_ML_ID'

In [3]:
%load_ext google.cloud.bigquery

## Update data

In [4]:

# List all tables in a dataset
tables = bq_client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'bqml-sandbox-396011.BQ_ML_ID':
bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_DATA


The target column name can be shortened <br>
and the marriage column should not be used in the prediction.

In [5]:
alter_table_sql_query = """
    ALTER TABLE `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_DATA`
    RENAME COLUMN default_payment_next_month TO card_default;

    ALTER TABLE `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_DATA`
    DROP COLUMN MARRIAGE;
"""

In [6]:
alter_table_query_job = bq_client.query(alter_table_sql_query)

alter_table_query_job.result()

## Create a training data

### Imbalanced data

The data is heavily imbalanced which is a common problem with Machine Learning. <br>
There many different ways to handle this problem <br>
One of the easiest ways are:
1. Resampling techniques,
2. Ensemble methods like Random Forest or Gradient  Boosting

The BigQuery ML does not have the function to perform oversampling or undersampling. <br>
Instead, the operations can be performed outside of the BigQuery (for example in Python) <br>
or create a procedure to perform this operations with help of Javascript https://cloud.google.com/bigquery/docs/user-defined-functions#supported-javascript-udf-data-types ,<br>
or the simplified version of random undersampling can be performed as shown below.

In [7]:
create_training_data_query = """
CREATE OR REPLACE TABLE `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_TRAINING_DATA`
AS
  (
    WITH CTE AS (
      SELECT *
      FROM `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_DATA`
      WHERE card_default = 0
      LIMIT 6000
    )

    SELECT * 
    FROM CTE

    UNION ALL
    SELECT *
    FROM `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_DATA`
    WHERE card_default = 1
  )
"""

In [8]:
create_training_data_query_job = bq_client.query(create_training_data_query)

create_training_data_query_job.result()

## Update schema for one-hot-encoding

### BigQuery ML automatic preprocessing

BigQuery ML can handle feature processing automatically. The process consists of two steps:
1. Missing value imputation,
2. Feature transformation

When it comes to imputation for example the NULLs in numeric columns are replaced with the mean value of a given column. <br>
While during the feature transformation, the numeric columns are standardised and other simple data types like BOOL or STRING are one-hot encoded. <br>
For most of the simple Machine Learning use cases, this type of automatic processing is quite sufficient. <br>

However, it is also possible to perform manual preprocessing using BiqQuery ML capabilities.<br>
Check out the documentation:
1. https://cloud.google.com/bigquery/docs/auto-preprocessing
2. https://cloud.google.com/bigquery/docs/manual-preprocessing

In [9]:
# Cast selected columns as strings to enable one-hot-encoding
update_schema_query = """
CREATE OR REPLACE TABLE `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_TRAINING_DATA` AS
SELECT 
      * EXCEPT(PAY_0,PAY_2,PAY_3,PAY_4,PAY_5, PAY_6),
      CAST(PAY_0 AS STRING) AS PAY_0,
      CAST(PAY_2 AS STRING) AS PAY_2,
      CAST(PAY_3 AS STRING) AS PAY_3,
      CAST(PAY_4 AS STRING) AS PAY_4,
      CAST(PAY_5 AS STRING) AS PAY_5,
      CAST(PAY_6 AS STRING) AS PAY_6

FROM `bqml-sandbox-396011.BQ_ML_ID.CUSTOMER_DEFAULT_TRAINING_DATA`
"""

In [10]:
update_schema_query_job = bq_client.query(update_schema_query)

update_schema_query_job.result()